# advent of code 2024 - [day 23](https://adventofcode.com/2024/day/23)


In [1]:
def gen_edges(file='input.txt'):
    file = open(file, 'r')
    for ix, line in enumerate(file):
        yield line.strip().split('-')

In [2]:
filename = 'input.txt'
#filename = 'test.txt'

In [ ]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']

from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [4]:
# cleaning
def clean():
    queries = [
    "CALL apoc.schema.assert({},{});",
    """MATCH (n)
    CALL {WITH n DETACH DELETE n}
    IN TRANSACTIONS OF 1000 ROWS;""",
    """CALL gds.graph.list()
    YIELD graphName
    WITH graphName AS g
    CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
    WITH graphName RETURN graphName;"""]

    for q in queries:
        gds.run_cypher(q, {})

In [5]:
def ingest(filename):

    links = [{'computer0': x[0], 'computer1': x[1]} for x in gen_edges(filename)]
    
    clean()

    gds.run_cypher('CREATE INDEX computer_id IF NOT EXISTS FOR (c:Computer) ON (c.id)')
    
    query_ingest = """
    UNWIND $links AS link
    MERGE (c0:Computer {id:link.computer0})
    MERGE (c1:Computer {id:link.computer1})
    MERGE (c0)-[:LINK]->(c1)
    """

    gds.run_cypher(query_ingest, {"links":links})

In [ ]:
ingest(filename)

In [ ]:
gds.run_cypher("""MATCH (c:Computer WHERE c.id STARTS WITH 't')
MATCH path = (c)-[:LINK]-(b)-[:LINK]-(a)-[:LINK]-(c)
WITH apoc.coll.toSet(apoc.coll.sort([x IN nodes(path) | x.id])) AS ids
RETURN count(DISTINCT ids) AS part1""")

In [ ]:
gds.run_cypher("""MATCH (source:Computer)
OPTIONAL MATCH (source)-[r:LINK]->(target:Computer)
RETURN gds.graph.project(
  'LAN',
  source,
  target,
  {
  },
  { undirectedRelationshipTypes: ['*'] }
)
""")

gds.run_cypher("""CALL gds.louvain.write('LAN', { writeProperty: 'community' })
YIELD communityCount, modularity, modularities""")

In [ ]:
gds.run_cypher("""MATCH (c:Computer)
WITH c, c.community AS community
WITH community, collect(c) AS cs
WITH community, cs, size(cs) AS sz ORDER BY sz DESC
MATCH edge = (a WHERE a IN cs)-[:LINK]->(b WHERE b IN cs)
WITH community, cs, count(edge) AS edges ORDER BY edges DESC
WHERE edges = (size(cs)-1) * (size(cs) -1 + 1) / 2 // check it's a clique
LIMIT 1
RETURN apoc.text.join(apoc.coll.toSet(apoc.coll.sort([x IN cs | x.id])),',') AS password""")